In [11]:
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
ty_df = pd.read_excel('../data/附件1.xlsx')
# sale_df = pd.read_excel('../data/附件2.xlsx')
purchase_df = pd.read_excel('../data/附件3.xlsx')
# decay_df = pd.read_excel('../data/附件4.xlsx')
ty_df.head(), purchase_df.head(), 
# sale_df.head(), decay_df.head()

(              单品编码    单品名称        分类编码 分类名称
 0  102900005115168    牛首生菜  1011010101  花叶类
 1  102900005115199   四川红香椿  1011010101  花叶类
 2  102900005115625  本地小毛白菜  1011010101  花叶类
 3  102900005115748     白菜苔  1011010101  花叶类
 4  102900005115762      苋菜  1011010101  花叶类,
           日期             单品编码  批发价格(元/千克)
 0 2020-07-01  102900005115762        3.88
 1 2020-07-01  102900005115779        6.72
 2 2020-07-01  102900005115786        3.19
 3 2020-07-01  102900005115793        9.24
 4 2020-07-01  102900005115823        7.03)

In [13]:
item_corpus = {row[1]['单品编码']: (row[1]['单品名称'], row[1]['分类编码']) 
               for row in ty_df.iterrows()}
ty_corpus = {row[1]['分类编码']: row[1]['分类名称'] 
             for row in ty_df.iterrows()}
print(list(item_corpus.items())[:5], list(ty_corpus.items())[:5], sep='\n')

[(102900005115168, ('牛首生菜', 1011010101)), (102900005115199, ('四川红香椿', 1011010101)), (102900005115625, ('本地小毛白菜', 1011010101)), (102900005115748, ('白菜苔', 1011010101)), (102900005115762, ('苋菜', 1011010101))]
[(1011010101, '花叶类'), (1011010201, '花菜类'), (1011010402, '水生根茎类'), (1011010501, '茄类'), (1011010504, '辣椒类')]


In [14]:
purchase_df['日期'] = pd.to_datetime(purchase_df['日期'])
purchase_df['年'] = purchase_df['日期'].dt.year
purchase_df['月'] = purchase_df['日期'].dt.month
purchase_df['日'] = purchase_df['日期'].dt.day
purchase_df.head()

,日期,单品编码,批发价格(元/千克),年,月,日
0,2020-07-01,102900005115762,3.88,2020,7,1
1,2020-07-01,102900005115779,6.72,2020,7,1
2,2020-07-01,102900005115786,3.19,2020,7,1
3,2020-07-01,102900005115793,9.24,2020,7,1
4,2020-07-01,102900005115823,7.03,2020,7,1


In [15]:
pch_grp = purchase_df.groupby(['单品编码'])
res = {}
month2season = {
    3: 0, 4: 0, 5: 0,
    6: 1, 7: 1, 8: 1,
    9: 2, 10: 2, 11: 2,
    12: 3, 1: 3, 2: 3
}
season_map = { 0: '春季', 1: '夏季', 2: '秋季', 3: '冬季' }
for idx, grp in pch_grp:
    uni_months = grp['月'].unique()
    season_list = [0] * 4
    seasons = []
    for month in uni_months:
        season_list[month2season[month]] += 1
        seasons.append(season_map[month2season[month]])
    num_seasons = 0
    for season in season_list:
        if season > 0:
            num_seasons += 1
    res[idx[0]] = {
        '月': sorted(uni_months),
        '月份数量': len(uni_months),
        '季节列表': season_list,
        '季节': seasons,
        '季节数量': num_seasons
    }
cnt = 0
for idx, row in res.items():
    print(f'{idx}: {row}')
    cnt += 1
    if cnt > 5:
        break

102900005115168: {'月': [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12], '月份数量': 11, '季节列表': [3, 3, 2, 3], '季节': ['秋季', '秋季', '冬季', '春季', '春季', '春季', '夏季', '夏季', '夏季', '冬季', '冬季'], '季节数量': 4}
102900005115199: {'月': [2, 3, 4, 5, 11, 12], '月份数量': 6, '季节列表': [3, 0, 1, 2], '季节': ['秋季', '冬季', '冬季', '春季', '春季', '春季'], '季节数量': 3}
102900005115250: {'月': [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12], '月份数量': 11, '季节列表': [3, 2, 3, 3], '季节': ['秋季', '秋季', '冬季', '冬季', '冬季', '春季', '春季', '夏季', '春季', '夏季', '秋季'], '季节数量': 4}
102900005115625: {'月': [9, 10, 11, 12], '月份数量': 4, '季节列表': [0, 0, 3, 1], '季节': ['秋季', '秋季', '秋季', '冬季'], '季节数量': 2}
102900005115748: {'月': [1, 2, 3, 5, 11, 12], '月份数量': 6, '季节列表': [2, 0, 1, 3], '季节': ['秋季', '冬季', '冬季', '春季', '冬季', '春季'], '季节数量': 3}
102900005115762: {'月': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], '月份数量': 11, '季节列表': [3, 3, 3, 2], '季节': ['夏季', '夏季', '秋季', '冬季', '冬季', '春季', '春季', '春季', '夏季', '秋季', '秋季'], '季节数量': 4}


In [16]:
cnt_all = 0
cnt_all_list = []
for k, v in res.items():
    if v['季节数量'] == 4:
        cnt_all += 1
        cnt_all_list.append(k)
cnt_all, cnt_all_list[:5]

(112,
 [102900005115168,
  102900005115250,
  102900005115762,
  102900005115779,
  102900005115786])

In [17]:
result = purchase_df.groupby(['单品编码', '年']).agg({'日期': 'nunique'}).reset_index()
result.rename(columns={'日期': '天数'}, inplace=True)
result.head()

,单品编码,年,天数
0,102900005115168,2020,4
1,102900005115168,2021,83
2,102900005115168,2022,21
3,102900005115168,2023,12
4,102900005115199,2020,13
